In [1]:
import sys
print(sys.executable)

C:\Users\fashi\anaconda3\envs\cityscapes\python.exe


In [2]:
import torch
print(torch.__version__)

2.5.1


In [ ]:
import os
import random
import shutil

# Define paths
image_dir = 'data'  # Folder containing all images
gt_dir = 'data2'     # Folder containing all ground truth files
holdout_dir = 'holdout_data'

# Create holdout directory if it doesn't exist
os.makedirs(holdout_dir, exist_ok=True)

# Get list of all images
all_images = [os.path.join(image_dir, file) for file in os.listdir(image_dir) if file.endswith('.png')]

# Randomly select 10% of the images (100 images out of 1000)
random.seed(42)  # For reproducibility
selected_images = random.sample(all_images, 100)

# Copy selected images and corresponding ground truth files to holdout directory
for img_path in selected_images:
    # Get the base filename (without _leftImg8bit.png)
    base_filename = os.path.basename(img_path).replace('_leftImg8bit.png', '')

    # Copy image
    shutil.copy(img_path, os.path.join(holdout_dir, os.path.basename(img_path)))

    # Copy corresponding ground truth files
    for suffix in ['_gtFine_color.png', '_gtFine_instanceIds.png', '_gtFine_labelIds.png', '_gtFine_polygons.json']:
        gt_file = os.path.join(gt_dir, base_filename + suffix)
        if os.path.exists(gt_file):
            shutil.copy(gt_file, os.path.join(holdout_dir, base_filename + suffix))

print("Holdout data created successfully.")

Holdout data created successfully.


In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.utils import save_image
from PIL import Image
import os

# Define FGSM attack
def fgsm_attack(image, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon * sign_data_grad
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

# Define PGD attack
def pgd_attack(image, epsilon, alpha, num_iter, data_grad):
    perturbed_image = image.clone()
    for _ in range(num_iter):
        perturbed_image = perturbed_image + alpha * data_grad.sign()
        eta = torch.clamp(perturbed_image - image, -epsilon, epsilon)
        perturbed_image = torch.clamp(image + eta, 0, 1)
    return perturbed_image

# Load an image
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    image = transform(image).unsqueeze(0)
    return image

# Save perturbed image
def save_perturbed_image(perturbed_image, output_path):
    save_image(perturbed_image, output_path)

# Apply attacks to holdout data
holdout_dir = 'holdout_data'  # Folder containing selected images and ground truth files
fgsm_dir = 'fgsm_contaminated'  # Folder to save FGSM-perturbed images
pgd_dir = 'pgd_contaminated'  # Folder to save PGD-perturbed images

os.makedirs(fgsm_dir, exist_ok=True)
os.makedirs(pgd_dir, exist_ok=True)

epsilon = 0.03  # Perturbation strength
alpha = 0.01  # Step size for PGD
num_iter = 10  # Number of iterations for PGD

# Process each image in the holdout directory
for img_file in os.listdir(holdout_dir):
    if img_file.endswith('_leftImg8bit.png'):  # Only process image files
        img_path = os.path.join(holdout_dir, img_file)
        image = load_image(img_path)
        image.requires_grad = True

        # Dummy loss (for demonstration purposes)
        output = image.mean()
        loss = -output  # Negative loss to maximize the perturbation
        loss.backward()
        data_grad = image.grad.data

        # Apply FGSM
        perturbed_image_fgsm = fgsm_attack(image, epsilon, data_grad)
        save_perturbed_image(perturbed_image_fgsm, os.path.join(fgsm_dir, img_file))

        # Apply PGD
        perturbed_image_pgd = pgd_attack(image, epsilon, alpha, num_iter, data_grad)
        save_perturbed_image(perturbed_image_pgd, os.path.join(pgd_dir, img_file))

print("Adversarial attacks applied successfully.")

Adversarial attacks applied successfully.
